Import file with Gmail App Password

In [8]:
import credentials

Add tickers of companies that I am interested in right now


In [9]:
tickers = ['TSLA','^GSPC', 'RIVN', 'AAPL', 'NVDA', 'DIS','MSFT','NFLX','LCID','CLOV','BUD','BB','PYPL','PLTR', 'FSR','AMD','COIN','^SPX']

In [10]:
#import all neccessary libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import timedelta
import datetime as dt

#get start and end days for the data download
start_date = (pd.Timestamp.today() - timedelta(days=7)).strftime('%Y-%m-%d')
end_date = (pd.Timestamp.today()).strftime('%Y-%m-%d')
day_of_week = (pd.Timestamp.today()).strftime('%A')
#prepare empty dataframes
prices_df = pd.DataFrame()
percentage_change_df = pd.DataFrame()
weekly_pct_change_df = pd.DataFrame()
h_prices_df = pd.DataFrame()

#loop to get yfinance data for all tickers, prepare it and then visualise and save
for ticker in tickers:
    #download and manipulate the data
    data = yf.download(ticker, start=start_date, end=end_date, actions=True)
    prices_df[ticker] = data['Adj Close']
    h_data = yf.download(ticker, start=start_date, end=end_date, actions=True, interval='5m')
    h_prices_df[ticker] = h_data['Adj Close'].reset_index(drop=True)
    percentage_change_df[ticker] = data['Adj Close'].pct_change() * 100
    weekly_data = (1 - (prices_df[:1].reset_index(drop=True) / prices_df[-1:].reset_index(drop=True))) * 100
    daily_data = (1 - (prices_df[-2:-1].reset_index(drop=True) / prices_df[-1:].reset_index(drop=True))) * 100

    datetime_index = pd.to_datetime(h_data.index)
    #get all of the days in the date range
    datetime_series = pd.Series(datetime_index)
    unique_dates = datetime_series.dt.strftime('%Y-%m-%d').unique()

    #create weekly price chart
    plt.figure(figsize=(10, 6))
    plt.plot(h_prices_df.index, h_prices_df[ticker], label=ticker)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Weekly Price Movement - {ticker}')
    plt.legend()

    #add vertical lines indicating different days 
    num_data_points = 390
    vertical_lines_indices = range(0, num_data_points, 78)
    for i,index in enumerate(vertical_lines_indices):
        plt.axvline(x=h_prices_df.index[index], color='gray', linestyle='--')
        #add text with dates to indicate the days
        plt.text(
                h_prices_df.index[index]+20,
                (h_prices_df[ticker]).min()-(h_prices_df[ticker].max() - h_prices_df[ticker].min())*0.01,
                unique_dates[i],
                rotation=0,
                va='top',
                ha='left',
                color='black'
            )


    plt.xticks(rotation=45)
    plt.xlim(0, h_prices_df.index[-1])
    plt.xticks([])

    plt.tight_layout()

    #save charts to png
    plt.savefig(f'{ticker}_weekly_price_movement.png')
    plt.close()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [11]:
daily_data = daily_data.copy().sort_index(axis=1)
weekly_data = weekly_data.copy().sort_index(axis=1)

In [12]:
#sort data to get best and worst weekly performers
t_weekly = weekly_data.transpose()
new_columns = ['Movement']
t_weekly.columns = new_columns
l_week = t_weekly.sort_values(by = 'Movement',ascending=True).head()
w_week = t_weekly.sort_values(by = 'Movement',ascending=False).head()

In [13]:
#sort data to get best and worst weekly performers
t_daily = daily_data.transpose()
new_column = ['Movement']
t_daily.columns = new_column
l_day = t_daily.sort_values(by = 'Movement',ascending=True).head()
w_day = t_daily.sort_values(by = 'Movement',ascending=False).head()

Prepare the Python and Html code to send and format emails

In [14]:
#import neccessary modules
import smtplib, ssl
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

#path to image that will be inserted into the email body
image_path = "^GSPC_weekly_price_movement.png"

#open and attach the image
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

image_part = MIMEImage(image_data)
image_cid = image_path
image_part.add_header("Content-ID", f"<{image_cid}>")
#start formatting the body of the email
#this part creates a 'column' of the best daily performers
best_daily = "<ul>\n"
for i in w_day.index:
    movement = round(w_day.loc[i,'Movement'],3)
    if movement >= 0:
        color = "green"
    else:
        color = "red"
    best_daily += f'<li><span>{i}: </span><span style="color:{color};">{movement}%</span></li>\n'

best_daily += "</ul>"
#repeat the process for the worst performers
worst_daily = "<ul>\n"
for i in l_day.index:
    movement = round(l_day.loc[i,'Movement'],3)
    if movement >= 0:
        color = "green"
    else:
        color = "red"
    worst_daily += f'<li><span>{i}: </span><span style="color:{color};">{movement}%</span></li>\n'

worst_daily += "</ul>"
#repeat the process on a weekly scale
best_weekly= "<ul>\n"
for i in w_week.index:
    movement = round(w_week.loc[i,'Movement'],3)
    if movement >= 0:
        color = "green"
    else:
        color = "red"
    best_weekly += f'<li><span>{i}: </span><span style="color:{color};">{movement}%</span></li>\n'

best_weekly += "</ul>"

worst_weekly = "<ul>\n"
for i in l_week.index:
    movement = round(l_week.loc[i,'Movement'],3)
    if movement >= 0:
        color = "green"
    else:
        color = "red"
    worst_weekly += f'<li><span>{i}: </span><span style="color:{color};">{movement}%</span></li>\n'

worst_weekly += "</ul>"


#repeat for all tickers for daily and weekly results, sorted alphabethically
daily_moves = "<ul>\n"
for column in daily_data.columns:
    movement = round(daily_data[column].iloc[-1],3)
    if movement >= 0:
        color = "green"
    else:
        color = "red"
    daily_moves += f'<li><span>{column}: </span><span style="color:{color};">{movement}%</span></li>\n'

daily_moves += "</ul>"

weekly_moves = "<ul>\n"
for column in weekly_data.columns:
    movement = round(weekly_data[column].iloc[-1],3)
    if movement >= 0:
        color = "green"
    else:
        color = "red"
    weekly_moves += f'<li><span>{column}: </span><span style="color:{color};">{movement}%</span></li>\n'

weekly_moves += "</ul>"


#create html tables so that columns with both best and worst performers can be seen next to each other
daily_table = f"""
<table style="width: 100%;">
    <tr>
        <td>
            <h3>Best Daily Performers:</h3>
            {best_daily}
        </td>
        <td>
            <h3>Worst Daily Performers:</h3>
            {worst_daily}
        </td>
    </tr>
</table>
"""
weekly_table = f"""
<table style="width: 100%;">
    <tr>
        <td>
            <h3>Best Weekly Performers:</h3>
            {best_weekly}
        </td>
        <td>
            <h3>Worst Weekly Performers:</h3>
            {worst_weekly}
        </td>
    </tr>
</table>
"""
all_table = f"""
<table style="width: 100%;">
    <tr>
        <td>
            <h3>Daily Movement:</h3>
            {daily_moves}
        </td>
        <td>
            <h3>Weekly Movement:</h3>
            {weekly_moves}
        </td>
    </tr>
</table>
"""

#finalise the email body woth the title in the body, embedded image and all of the tables inserted
html_content = f"""
<!DOCTYPE html>
<html>
    <h1>Your daily stock market report</h1>
    <img src="cid:{image_cid}" alt="S&P 500 Weekly Price Movement">
    {daily_table}
    {weekly_table}
    {all_table}
</body>
</html>
"""

#set up the neccessary credentials to send the emails
email_from = credentials.mail_address
password = credentials.mail_password
email_to = credentials.mail_address

#define the sender,reciever and email title
email_message = MIMEMultipart()
email_message['From'] = email_from
email_message['To'] = email_to
email_message['Subject'] = f'Report email - {day_of_week}, {end_date}'

#'attach' the html body to the email
email_message.attach(MIMEText(html_content, "html"))
#define the function to add all of the time series graphs for each ticker as as png attachments
def attach_file_to_email(email_message, filename):
    with open(filename, "rb") as f:
        file_attachment = MIMEApplication(f.read())
    file_attachment.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}")
    # Attach the file to the message
    email_message.attach(file_attachment)
tiskers = tickers.copy().sort()
#loop through tickers and attach all of the timeseries visualisations
for ticker in tickers:
    filename = f"{ticker}_weekly_price_movement.png"
    attach_file_to_email(email_message, filename)


email_message.attach(image_part)

#connect with smtp server and send the finalised email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(email_from, password)
    server.sendmail(email_from, email_to, email_message.as_string())
